# GRIP Task 7


# Stock Market Prediction using Numerical and Textual Analysis

# Objective: Create a hybrid model for stock price/performance prediction using numerical analysis of historical stock prices, and sentimental analysis of news headlines

# Jasmine Boro

In [9]:
!pip install keras

In [11]:
!pip install tensorflow

  Stored in directory: C:\Users\Jasmine\AppData\Local\pip\Cache\wheels\7c\06\54\bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Stored in directory: C:\Users\Jasmine\AppData\Local\pip\Cache\wheels\b1\c2\ed\d62208260edbd3fa7156545c00ef966f45f2063d0a84f8208a
Successfully built termcolor wrapt
  Found existing installation: six 1.12.0
    Uninstalling six-1.12.0:
      Successfully uninstalled six-1.12.0
  Found existing installation: wheel 0.33.1
    Uninstalling wheel-0.33.1:
      Successfully uninstalled wheel-0.33.1
  Found existing installation: setuptools 40.8.0
    Uninstalling setuptools-40.8.0:
      Successfully uninstalled setuptools-40.8.0
  Found existing installation: h5py 2.9.0
    Uninstalling h5py-2.9.0:
      Successfully uninstalled h5py-2.9.0
  Found existing installation: wrapt 1.11.1
    Uninstalling wrapt-1.11.1:
      Successfully uninstalled wrapt-1.11.1


tables 3.5.1 requires mock>=2.0, which is not installed.
astroid 2.2.5 requires typed-ast>=1.3.0; implementation_name == "cpython", which is not installed.


In [12]:
# importing libraries
import warnings
warnings.filterwarnings('ignore')

import math
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing, metrics
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Dense, Activation

import nltk
nltk.download('vader_lexicon')
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Jasmine\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [18]:
!pip install yfinance

  Stored in directory: C:\Users\Jasmine\AppData\Local\pip\Cache\wheels\fe\a0\79\b73d4a0c535b421b88fc7b393936b371fabbfeaf979eca4050
  Stored in directory: C:\Users\Jasmine\AppData\Local\pip\Cache\wheels\37\fa\73\d492849e319038eb4d986f5152e4b19ffb1bc0639da84d2677
Successfully built yfinance multitasking
  Found existing installation: lxml 4.3.2
    Uninstalling lxml-4.3.2:
      Successfully uninstalled lxml-4.3.2


In [19]:
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
import io

In [26]:
!pip install yahoofinancials

  Stored in directory: C:\Users\Jasmine\AppData\Local\pip\Cache\wheels\d9\7e\cf\4977a8572d5247242a4b13018d1d36923024ba84236e0d28bc
Successfully built yahoofinancials


In [27]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials

In [29]:
teslastockprice = yf.download('TSLA', 
                      start='2019-01-01', 
                      end='2019-12-31', 
                      progress=False)
teslastockprice.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-12-31,67.557999,67.842003,65.052002,66.559998,66.559998,31511500
2019-01-02,61.220001,63.026001,59.759998,62.023998,62.023998,58293000
2019-01-03,61.400002,61.880001,59.476002,60.071999,60.071999,34826000
2019-01-04,61.200001,63.599998,60.546001,63.537998,63.537998,36970500
2019-01-07,64.344002,67.348000,63.549999,66.991997,66.991997,37756000


In [30]:
stockheadline=pd.read_csv(r"C:\Users\Jasmine\Downloads\india-news-headlines.csv")
stockheadline.head()

,publish_date,headline_category,headline_text
0,20010102,unknown,Status quo will not be disturbed at Ayodhya; s...
1,20010102,unknown,Fissures in Hurriyat over Pak visit
2,20010102,unknown,America's unwanted heading for India?
3,20010102,unknown,For bigwigs; it is destination Goa
4,20010102,unknown,Extra buses to clear tourist traffic


In [31]:
len(teslastockprice),len(stockheadline)

(252, 3424067)

In [32]:
teslastockprice.isna().any(), stockheadline.isna().any()

(Open         False
 High         False
 Low          False
 Close        False
 Adj Close    False
 Volume       False
 dtype: bool, publish_date         False
 headline_category    False
 headline_text        False
 dtype: bool)

# Stock Price

In [40]:
# dropping duplicates in data
teslastockprice = teslastockprice.drop_duplicates()
# sorting the data according to Date
teslastockprice = teslastockprice.sort_index(ascending=True, axis=0)
teslastockprice.head()


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-12-31,67.557999,67.842003,65.052002,66.559998,66.559998,31511500
2019-01-02,61.220001,63.026001,59.759998,62.023998,62.023998,58293000
2019-01-03,61.400002,61.880001,59.476002,60.071999,60.071999,34826000
2019-01-04,61.200001,63.599998,60.546001,63.537998,63.537998,36970500
2019-01-07,64.344002,67.348000,63.549999,66.991997,66.991997,37756000


# Stock Headlines

In [55]:
# dropping duplicates
stockheadline = stockheadline.drop_duplicates()
# filtering the important columns 
stockheadline = stockheadline.filter(['publish_date', 'headline_text'])
# grouping the news headlines according to Date
stockheadline = stockheadline.groupby(['publish_date'])['headline_text'].apply(lambda x: ','.join(x)).reset_index()
# setting Date as the index column
stockheadline.set_index('publish_date', inplace= True)
# sorting the data according to the Date
stockheadline = stockheadline.sort_index(ascending=True, axis=0)
stockheadline.head()


,headline_text
publish_date,
2001-01-01,Status quo will not be disturbed at Ayodhya; s...
2002-01-01,"Will 2002 be the year of the bug?,State to be ..."
2003-01-01,"Chastened BCC members decide to mend ways,New ..."
2004-01-01,"Try this: Emu as year-end grill,Airport work m..."
2005-01-01,"The Bottom Line,A New Year Gift,She beat death..."
